In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [ ]:


def detect_ball_color(frame, ball_type):
    """
    Detect ball using color segmentation
    """
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define color ranges for different balls
    color_ranges = {
        'white': ([0, 0, 180], [180, 50, 255]),      # Bright white
        'blue': ([100, 150, 50], [130, 255, 255]),   # Blue
        'black': ([0, 0, 0], [180, 255, 80]),        # Dark black
        'red': ([0, 120, 70], [10, 255, 255]),       # Red
        'yellow': ([20, 100, 100], [30, 255, 255]),  # Yellow
        'green': ([40, 100, 100], [80, 255, 255]),   # Green
        'pink': ([140, 100, 100], [170, 255, 255])   # Pink
    }
    
    lower, upper = color_ranges.get(ball_type, ([0, 0, 0], [180, 255, 255]))
    mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
    
    # Clean up the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    return mask

def track_ball_with_circle(video_path, ball_type='black', output_video='tracked_output.mp4'):
    """
    Track ball and draw circle around it WITHOUT removing background
    """
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return False
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Video: {width}x{height}, FPS: {fps:.2f}")
    
    # Setup video writer - SAVES OUTPUT
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))
    
    trajectory = []
    frame_count = 0
    
    print(f"Tracking {ball_type} ball...")
    print("Press 'q' to quit early")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video reached")
            break
        
        # Create display frame (keep original background)
        display_frame = frame.copy()
        
        # Detect ball using color
        mask = detect_ball_color(frame, ball_type)
        
        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        ball_detected = False
        ball_position = None
        ball_radius = 0
        
        if contours:
            # Find the largest contour that could be our ball
            largest_contour = max(contours, key=cv2.contourArea)
            area = cv2.contourArea(largest_contour)
            
            # Filter by area (adjust these values based on your video)
            if 50 < area < 3000:  # Ball size range
                # Check circularity to ensure it's ball-like
                perimeter = cv2.arcLength(largest_contour, True)
                if perimeter > 0:
                    circularity = 4 * np.pi * area / (perimeter * perimeter)
                    
                    # Good circle has circularity close to 1.0
                    if circularity > 0.5:  
                        (x, y), radius = cv2.minEnclosingCircle(largest_contour)
                        ball_position = (int(x), int(y))
                        ball_radius = int(radius)
                        ball_detected = True
        
        # DRAW ON ORIGINAL FRAME (NO BACKGROUND REMOVAL)
        if ball_detected and ball_radius > 5:
            # Draw YELLOW circle around the ball (as you requested)
            cv2.circle(display_frame, ball_position, ball_radius, (0, 255, 255), 2)
            
            # Draw RED center dot
            cv2.circle(display_frame, ball_position, 3, (0, 0, 255), -1)
            
            # Add to trajectory
            trajectory.append(ball_position)
            
            # Draw trajectory (last 30 points) with BLUE lines
            for i in range(1, min(len(trajectory), 30)):
                if trajectory[i-1] is not None and trajectory[i] is not None:
                    cv2.line(display_frame, trajectory[i-1], trajectory[i], (255, 0, 0), 2)
            
            # Display ball info
            info_text = f"{ball_type.upper()} Ball: ({ball_position[0]}, {ball_position[1]})"
            cv2.putText(display_frame, info_text, 
                       (ball_position[0] - 40, ball_position[1] - ball_radius - 15),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
        
        # SAVE FRAME TO OUTPUT VIDEO (with circles but original background)
        out.write(display_frame)
        
        # Display progress
        display_frame_small = cv2.resize(display_frame, (width//2, height//2))
        cv2.imshow('Ball Tracking - Original Background', display_frame_small)
        
        frame_count += 1
        if frame_count % 30 == 0:
            print(f"Processed frame {frame_count} - Ball detected: {ball_detected}")
        
        # Press 'q' to quit early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Stopped by user")
            break
    
    # Clean up
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    print(f"\nSUCCESS! Tracking completed!")
    print(f" Output saved as: {output_video}")
    print(f"Frames processed: {frame_count}")
    print(f" Trajectory points: {len(trajectory)}")
    
    return True

# USAGE - SIMPLE AND CLEAN
if __name__ == "__main__":
    # CHANGE THIS TO YOUR VIDEO PATH
    VIDEO_PATH = "./test.mp4"
    
    # CHOOSE BALL COLOR: 'white', 'blue', 'black', 'red', 'yellow', 'green', 'pink'
    BALL_COLOR = "black"  
    
    # Output filename
    OUTPUT_FILE = f"tracked_{BALL_COLOR}_ball_with_circle.mp4"
    
    # Run tracking
    success = track_ball_with_circle(
        video_path=VIDEO_PATH,
        ball_type=BALL_COLOR,
        output_video=OUTPUT_FILE
    )
    
    if success:
        print(f"\n🎉 Check your folder for: {OUTPUT_FILE}")
        print("The video shows the original background with yellow circles around the ball!")
    else:
        print("❌ Tracking failed. Check your video path.")

Video: 640x360, FPS: 30.00
Tracking black ball...
Press 'q' to quit early
Processed frame 30 - Ball detected: False
Processed frame 60 - Ball detected: False
Processed frame 90 - Ball detected: False
Processed frame 120 - Ball detected: False
Processed frame 150 - Ball detected: False
Processed frame 180 - Ball detected: False
Processed frame 210 - Ball detected: False
Processed frame 240 - Ball detected: False
Processed frame 270 - Ball detected: False
Processed frame 300 - Ball detected: False
Processed frame 330 - Ball detected: False
Processed frame 360 - Ball detected: False
Processed frame 390 - Ball detected: False
Processed frame 420 - Ball detected: False
Processed frame 450 - Ball detected: False
Processed frame 480 - Ball detected: False
Processed frame 510 - Ball detected: False
Processed frame 540 - Ball detected: False
Processed frame 570 - Ball detected: False
Processed frame 600 - Ball detected: False
Processed frame 630 - Ball detected: False
Processed frame 660 - Ball

In [ ]:

width, height = 280, 560

def detect_blue_ball_red_table(transformed_img):
    """
    Detects the BLUE ball on RED pool table
    """
    blurred = cv2.GaussianBlur(transformed_img, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    # Blue color range
    lower_blue = np.array([90, 90, 50])
    upper_blue = np.array([130, 255, 255])

    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

    kernel = np.ones((7, 7), np.uint8)
    mask_blue = cv2.morphologyEx(mask_blue, cv2.MORPH_CLOSE, kernel)
    mask_blue = cv2.GaussianBlur(mask_blue, (5, 5), 0)

    contours, _ = cv2.findContours(mask_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    blue_ball = None
    max_score = 0

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 30 < area < 900:  
            perimeter = cv2.arcLength(cnt, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            score = circularity * area
            if score > max_score:
                max_score = score
                blue_ball = cnt

    return blue_ball, mask_blue

def detect_black_ball_red_table(transformed_img):
    """
    Detects BLACK ball on red table using darkness
    """
    blurred = cv2.GaussianBlur(transformed_img, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    # black one
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 70])
    
    mask_black = cv2.inRange(hsv, lower_black, upper_black)
    
    kernel = np.ones((7, 7), np.uint8)
    mask_black = cv2.morphologyEx(mask_black, cv2.MORPH_OPEN, kernel)
    mask_black = cv2.morphologyEx(mask_black, cv2.MORPH_CLOSE, kernel)
    
    contours, _ = cv2.findContours(mask_black, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    black_ball = None
    max_score = 0
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 30 < area < 900:
            perimeter = cv2.arcLength(cnt, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            score = circularity * area
            if score > max_score:
                max_score = score
                black_ball = cnt
    
    return black_ball, mask_black

def detect_white_ball_red_table(transformed_img):
    """
    Detects WHITE ball on red table using brightness
    """
    blurred = cv2.GaussianBlur(transformed_img, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    # white one
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 50, 255])
    
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    
    kernel = np.ones((5, 5), np.uint8)
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_CLOSE, kernel)
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_OPEN, kernel)
    
    contours, _ = cv2.findContours(mask_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    white_ball = None
    max_score = 0
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 30 < area < 900:
            perimeter = cv2.arcLength(cnt, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            score = circularity * area
            if score > max_score:
                max_score = score
                white_ball = cnt
    
    return white_ball, mask_white

def track_ball_two_outputs(video_path, ball_type='black', 
                           output_tracked='tracked_with_circle.mp4',
                           output_ballonly='ball_only_blackbg.mp4'):

    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return
    

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    orig_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    orig_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Input video: {video_path}")
    print(f"Size: {orig_w}x{orig_h}, FPS: {fps:.2f}, Frames: {total_frames}")
    
 
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_tracked = cv2.VideoWriter(output_tracked, fourcc, fps, (orig_w, orig_h))
    out_ballonly = cv2.VideoWriter(output_ballonly, fourcc, fps, (orig_w, orig_h))
 
    pts1 = np.float32([[255, 60], [590, 60], [160, 380], [690, 380]])
    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])

    
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    
    frame_num = 0
    trajectory = []  
    
    print(f"Tracking {ball_type.upper()} ball...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
      
        transformed = cv2.warpPerspective(frame, matrix, (width, height))
        
       
        if ball_type == "blue":
            ball_contour, ball_mask = detect_blue_ball_red_table(transformed)
        elif ball_type == "black":
            ball_contour, ball_mask = detect_black_ball_red_table(transformed)
        else:  
            ball_contour, ball_mask = detect_white_ball_red_table(transformed)
        
   
        tracked_frame = frame.copy()
        
       
        ball_only_frame = np.zeros_like(frame)
        
        if ball_contour is not None:
          
            M = cv2.moments(ball_contour)
            if M['m00'] != 0:
                cX_trans = int(M['m10'] / M['m00'])
                cY_trans = int(M['m01'] / M['m00'])
                
                (x, y), radius = cv2.minEnclosingCircle(ball_contour)
                radius = int(radius)
                

                pts_transformed = np.array([[[cX_trans, cY_trans]]], dtype=np.float32)
                inv_matrix = cv2.getPerspectiveTransform(pts2, pts1)
                pts_original = cv2.perspectiveTransform(pts_transformed, inv_matrix)
                
                cX_orig = int(pts_original[0][0][0])
                cY_orig = int(pts_original[0][0][1])
                
                radius_orig = int(radius * (orig_w / width))
                
               
                trajectory.append((cX_orig, cY_orig))
                
                cv2.circle(tracked_frame, (cX_orig, cY_orig), radius_orig, (0, 255, 0), 2)
                cv2.circle(tracked_frame, (cX_orig, cY_orig), 2, (0, 0, 255), -1)
               
                for i in range(1, len(trajectory)):
                    cv2.line(tracked_frame, trajectory[i-1], trajectory[i], (0, 255, 255), 2)
                
           
                mask_orig = np.zeros((orig_h, orig_w), dtype=np.uint8)
                cv2.circle(mask_orig, (cX_orig, cY_orig), max(1, radius_orig - 2), 255, -1)
                
         
                for c in range(3):
                    ball_only_frame[:, :, c] = cv2.bitwise_and(frame[:, :, c], frame[:, :, c], mask=mask_orig)

        out_tracked.write(tracked_frame)
        out_ballonly.write(ball_only_frame)

        if frame_num % 30 == 0:
            print(f"Processing frame {frame_num}/{total_frames}")
        
        frame_num += 1

        display_scale = 0.5
        tracked_display = cv2.resize(tracked_frame, None, fx=display_scale, fy=display_scale)
        ballonly_display = cv2.resize(ball_only_frame, None, fx=display_scale, fy=display_scale)
        
        cv2.imshow('Tracked Video', tracked_display)
        cv2.imshow('Ball Only', ballonly_display)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out_tracked.release()
    out_ballonly.release()
    cv2.destroyAllWindows()
    
    print(f"\n  DONE! Two videos created:")
    print(f"1. Tracked video: {output_tracked}")
    print(f"2. Ball only video: {output_ballonly}")
    print(f"Frames processed: {frame_num}")

if __name__ == "__main__":
    import os
    
    VIDEO_IN = "./test.mp4" #put your video path CTRL+ALT+C
    
    BALL_COLOR = "black"

    VIDEO_OUT_TRACKED = f"tracked_{BALL_COLOR}_ball.mp4"
    VIDEO_OUT_BALLONLY = f"{BALL_COLOR}_ball_only_blackbg.mp4"
    if not os.path.exists(VIDEO_IN):
        print(f"ERROR: Video file '{VIDEO_IN}' not found!")
        print(f"Current directory: {os.getcwd()}")
    else:
        track_ball_two_outputs(VIDEO_IN, 
                              ball_type=BALL_COLOR,
                              output_tracked=VIDEO_OUT_TRACKED,
                              output_ballonly=VIDEO_OUT_BALLONLY)

Input video: ./test.mp4
Size: 640x360, FPS: 30.00, Frames: 3300
Tracking BLACK ball...
Processing frame 0/3300
Processing frame 30/3300
Processing frame 60/3300
Processing frame 90/3300
Processing frame 120/3300
Processing frame 150/3300
Processing frame 180/3300
Processing frame 210/3300
Processing frame 240/3300
Processing frame 270/3300
Processing frame 300/3300
Processing frame 330/3300
Processing frame 360/3300
Processing frame 390/3300
Processing frame 420/3300
Processing frame 450/3300
Processing frame 480/3300
Processing frame 510/3300
Processing frame 540/3300
Processing frame 570/3300
Processing frame 600/3300
Processing frame 630/3300
Processing frame 660/3300
Processing frame 690/3300
Processing frame 720/3300
Processing frame 750/3300
Processing frame 780/3300
Processing frame 810/3300
Processing frame 840/3300
Processing frame 870/3300
Processing frame 900/3300
Processing frame 930/3300
Processing frame 960/3300
Processing frame 990/3300
Processing frame 1020/3300
Process